In [1]:
import pandas as pd
import pickle as pkl

In [2]:
def sign_bio_model_nodes(edges, all_nodes, node_num):
    '''
    edges: It is a dataframe with three columns: 'froms', 'to' and 'sign' where 'from' and 'to' corresponds 
    to vector of nodes from which an edge emanates and terminates respectively and sign denotes the nature of the edge.
    all_nodes: This is a list of nodes ([gene0, gene1, gene2,...])
    node_num: This is a dictionary that links genes with an index ({'gene0': 0, 'gene1': 1, 'gene2': 2, 'gene3': 3,
    'gene4': 4, 'gene5': 5, ...})
    returns information about the sign of the interactions in the network
        -({0: 'ai', 1: 'aia', 2: 'aaaaa', 3: 'a', 4: 'a', ...})
    '''
    signs = dict()
    for node in all_nodes:
        inter = edges.loc[edges["to"] == node, ["sign"]]
        signs[node_num[node]] = ''.join(list(inter.sign))
    return signs

In [6]:
def give_constrained_threshold_models(edges, in_edges, node_num):
    TMR_dict = {}
    for func in ['IMR', 'BMR']:
        all_nodes = list(node_num.keys())
        n = len(all_nodes)
        signs = sign_bio_model_nodes(edges, all_nodes, node_num)
        if func == 'IMR':
            self_loop_index_dict = {n: in_edges[n].index(n) for n in in_edges if len(in_edges[n]) % 2 == 0 and n in in_edges[n]}
            model = {}
            for i in range(n):
                orig_inp_number = len(in_edges[i])
                df1 = pd.read_csv(fr"../../../computational/all_IMR_tsv/all_IMR_k{orig_inp_number}.tsv", sep = '\t', dtype={"self_loop_index": str, "BF": str})
                if i in self_loop_index_dict:
                    bin_model = df1.loc[(df1["sign"] == signs[i]) & (df1["self_loop_index"] == str(self_loop_index_dict[i])), "BF"].astype(str).values[0]
                    model[i] = int(bin_model, 2)
                else:
                    bin_model = df1.loc[(df1["sign"] == signs[i]) & (df1["self_loop"] == 'N'), "BF"].astype(str).values[0]
                    model[i] = int(bin_model, 2)
            maj_rule_mod = list(model.values())
            TMR_dict[func] = maj_rule_mod
            
        if func == 'BMR':
            self_loop_index_dict = {n: in_edges[n].index(n) for n in in_edges if n in in_edges[n]}
            model = {}
            for i in range(n):
                orig_inp_number = len(in_edges[i])
                df1 = pd.read_csv(fr"../../../computational/all_BMR_tsv/all_BMR_k{orig_inp_number}.tsv", sep = '\t', dtype={"self_loop_index": str, "BF": str})
                if i in self_loop_index_dict:
                    bin_model = df1.loc[(df1["sign"] == signs[i]) & (df1["self_loop_index"] == str(self_loop_index_dict[i])), "BF"].astype(str).values[0]
                    model[i] = int(bin_model, 2)
                else:
                    bin_model = df1.loc[(df1["sign"] == signs[i]) & (df1["self_loop"] == 'N'), "BF"].astype(str).values[0]
                    model[i] = int(bin_model, 2)
            maj_rule_mod = list(model.values())
            TMR_dict[func] = maj_rule_mod
            
    return TMR_dict

In [9]:
mod_num = 10
edges = pd.read_csv(f'../../network_generation/output/published_network_info/model_{mod_num}/model_{mod_num}_edgelist.tsv', sep = '\t')
with open(f'../../network_generation/output/published_network_info/model_{mod_num}/inedges.pkl', 'rb') as file1:
    in_edges = pkl.load(file1)

with open(f'../../network_generation/output/published_network_info/model_{mod_num}/node_num_swap.pkl', 'rb') as file2:
    node_num_swap = pkl.load(file2)
    
    
node_num = {value: key for key, value in node_num_swap.items()}
A = give_constrained_threshold_models(edges, in_edges, node_num)
print(A)

{'IMR': [113, 77, 43, 23, 23, 18290559, 77, 5214401853277568295361350912147455, 1, 3653461613885928115, 1, 1, 1, 1, 1], 'BMR': [113, 29175, 19935, 127, 32767, 9223372036854775807, 77, 57896044618658097711785492504343953926634992332820282019728792003956564819967, 7, 3653461613885928115, 7, 7, 7, 1, 1]}
